In [1]:
#Need this location for rest of the project to be located from
import sys
sys.path.append('C:\\Users\\PAT\\Documents\\edwisor\\projects\\traffic_crash')

In [2]:
#Load the libraries from package 'traffic_crash/__init__.py file under the location mentioned above'
from traffic_crash import * 
from traffic_crash.config import *
from utils import *
from utils.utils_ import *
from utils.utils_local import *

### open the geojson file in to tmp and then create a pandas dataframe

In [3]:
f = open(json_data_path)
tmp = geojson.load(f)

#### pandas dataframe

In [4]:
data = pd.DataFrame()
crash = {}
sp = []
count = 0
for record in tmp:
    for key in record.keys():
        #if key == 'geo_shape':
        #    print(type(record[key])) 
        if type(record[key]) in [dict,geojson.feature.Feature]:
            if key == 'geo_shape':
                pass#Ignoring
            elif key == 'geo_point_2d':
                crash['geo_coord_lat'] = record[key]['lat']
                crash['geo_coord_lon'] = record[key]['lon']
            if key not in sp:
                sp.append(key)
                
        else:
            crash[key] = record[key]
    #print(crash)       
    each_crash = pd.DataFrame(data = crash,index=[count])
    count+=1
    data = pd.concat([each_crash,data],axis=0)        
data = data.reset_index()
data = data.drop('index',axis=1)

### check the data type of the dataframe columns

In [5]:
continous_names, cat_names, iv,dv,testparams = iv_dv_get_col_names(data,'ped_sex')
print('continous :',continous_names, '\ncategorical :',cat_names)

$$ adding ambulancer(object) as categorical
$$ adding rd_defects(object) as categorical
$$ adding crsh_sevri(object) as categorical
$$ adding objectid(object) as categorical
$$ adding crash_ty_1(object) as categorical
$$ adding crash_time(object) as categorical
$$ adding crash_year(object) as categorical
$$ adding county(object) as categorical
$$ adding speed_limi(object) as categorical
$$ adding rural_urba(object) as categorical
************************you may need to define this col - longitude in categ variable
******* adding longitude(float64) as continous as per the data type
$$ adding crash_mont(object) as categorical
$$ adding drvr_vehty(object) as categorical
$$ adding crash_type(object) as categorical
$$ adding city(object) as categorical
$$ adding workzone_i(object) as categorical
$$ adding objectid_1(float64) as continous
$$ adding severity(object) as categorical
$$ adding locality(object) as categorical
$$ adding crash_grp0(object) as categorical
$$ adding crashalcoh(object

In [6]:
data.loc[0,'latitude'],data.loc[0,'longitude'],data.loc[0,'geo_coord_lat'],data.loc[0,'geo_coord_lon']

(36.0, -79.0, 35.90427709037941, -79.0879027517705)

### Bird's eye view: Observations about data:
- Looks like longitude and latitude values are round off from geo_point_2d and geo_shape, So we can actually remove the redundant ones
- A lot of categorical data needs cleaning
    - crash_date (along with crash_time) needs to be converted to timestamp object and then stored
        - crash month, crash time are redundant, once we have all the data consolidated in timestamp object
- Longitude and latitude can be stored as float data type, as we need to calculate distance using those
- drv_age can be stored as int (for easier mathematical calculations) and later we can make it categorical if needed to analyze something
- Store longitude and latitude in geography data type(postgis) in psql database(for calculation ease)

### converting crash time,year,month,hour to a timestamp object

In [7]:
bk = data.copy()

In [8]:
#data = bk.copy()

In [9]:
month = {'JAN':1,'FEB':2,'MAR':3,'APR':4,'MAY':5,'JUN':6,'JUL':7,'AUG':8,'SEP':9,'OCT':10,'NOV':11,'DEC':12}
data['crash_timestamp'] = ''#pd.Timestamp(1, 1, 1, 1)
for index,row in enumerate(data):
    #print(data.loc[index,'crash_time'],data.loc[index,'crash_hour'])
    year_ = int(data.loc[index,'crash_year'])
    day_ = int(data.loc[index,'crash_date'][:2])
    month_ = month[(data.loc[index,'crash_date'][2:5])]
    hour_ = int(data.loc[index,'crash_time'].split(":")[0])
    min_ = int(data.loc[index,'crash_time'].split(":")[1])
    #print(hour_,min_)
    data.loc[index,'crash_timestamp'] = pd.Timestamp(year=year_,month=month_,day = day_,hour=hour_,minute=min_)
data = data.drop(['crash_year','crash_date','crash_date','crash_hour','crash_time'],axis=1)    

In [10]:
data.loc[data.crash_timestamp=='','crash_timestamp'] = 'EPOCH'

In [11]:
data[data.crash_timestamp=='EPOCH']

,ambulancer,rd_defects,crsh_sevri,objectid,crash_ty_1,county,speed_limi,rural_urba,longitude,crash_mont,...,crsh_typex,weather,num_units,crash_grp,num_lanes,hit_run,drvr_estsp,rd_config,ped_sex,crash_timestamp
65,Yes,None,B: Evident Injury,997,38830,Durham,5 - 15 MPH,Urban,-79.0,June,...,None,Rain,2.0,Off Roadway,Unknown,No,16-20 mph,"Two-Way, Not Divided",Male,EPOCH
66,Yes,None,B: Evident Injury,1906,37890,Orange,5 - 15 MPH,Urban,-79.0,March,...,None,Clear,2.0,Off Roadway,1 lane,Yes,Unknown,"One-Way, Not Divided",Male,EPOCH
67,Yes,None,C: Possible Injury,1836,38830,Orange,Unknown,Urban,-79.0,December,...,None,Clear,2.0,Off Roadway,Unknown,No,0-5 mph,"Two-Way, Not Divided",Male,EPOCH
68,Yes,None,B: Evident Injury,1485,23760,Orange,30 - 35 MPH,Urban,-79.0,September,...,None,Clear,2.0,Crossing Roadway - Vehicle Not Turning,5 lanes,No,26-30 mph,"Two-Way, Not Divided",Female,EPOCH
69,Yes,None,B: Evident Injury,2039,3110,Orange,5 - 15 MPH,Urban,-79.0,November,...,None,Clear,3.0,Unusual Circumstances,Unknown,No,Unknown,Unknown,Male,EPOCH
70,Yes,None,B: Evident Injury,1975,13760,Orange,30 - 35 MPH,Urban,-79.0,August,...,None,Clear,2.0,Crossing Roadway - Vehicle Not Turning,2 lanes,No,21-25 mph,"Two-Way, Not Divided",Male,EPOCH
71,No,Other,C: Possible Injury,1737,23781,Orange,5 - 15 MPH,Urban,-79.0,January,...,None,Rain,2.0,Crossing Roadway - Vehicle Turning,4 lanes,No,0-5 mph,"Two-Way, Not Divided",Female,EPOCH
72,No,None,O: No Injury,1768,12781,Orange,20 - 25 MPH,Urban,-79.0,January,...,None,Clear,2.0,Crossing Roadway - Vehicle Turning,4 lanes,No,6-10 mph,"Two-Way, Divided, Unprotected Median",Female,EPOCH
73,Yes,None,B: Evident Injury,1984,23760,Durham,30 - 35 MPH,Urban,-79.0,January,...,None,Clear,2.0,Crossing Roadway - Vehicle Not Turning,2 lanes,No,36-40 mph,"Two-Way, Not Divided",Male,EPOCH
74,Yes,Road Surface Condition,B: Evident Injury,1913,43620,Orange,20 - 25 MPH,Urban,-79.0,January,...,None,"Snow, Sleet, Hail, Freezing Rain/Drizzle",2.0,Pedestrian in Roadway - Circumstances Unknown,2 lanes,No,11-15 mph,"Two-Way, Not Divided",Male,EPOCH


In [12]:
data.head(3)

,ambulancer,rd_defects,crsh_sevri,objectid,crash_ty_1,county,speed_limi,rural_urba,longitude,crash_mont,...,crsh_typex,weather,num_units,crash_grp,num_lanes,hit_run,drvr_estsp,rd_config,ped_sex,crash_timestamp
0,No,None,None,1254,None,Orange,30 - 35 MPH,Urban,-79.0,June,...,38830,Clear,2.0,Off Roadway,Unknown,Yes,16-20 mph,"Two-Way, Not Divided",Female,2012-06-09 20:37:00
1,Yes,None,None,734,None,Orange,20 - 25 MPH,Urban,-79.0,March,...,22770,Clear,2.0,Crossing Roadway - Vehicle Not Turning,Unknown,No,31-35 mph,"Two-Way, Not Divided",Female,2012-03-26 15:11:00
2,No,None,C: Possible Injury,1908,23680,Durham,5 - 15 MPH,Urban,-79.0,October,...,None,Clear,2.0,Other / Unknown - Insufficient Details,4 lanes,No,0-5 mph,"Two-Way, Not Divided",Female,2010-10-11 13:28:00


####  keep only lat and long obtained from geo_point_2d in the json, as others(longitude,latitude,geo_shape) are redundant

In [13]:
data = data.drop(['latitude','longitude'],axis=1)

In [14]:
data.shape

(320, 59)

##  Find all crashes that happened between the 
- timing 8 pm and 8:40 pm in any location 
- which is within 11 km radius of coordinates  [-78.95570271414434, 35.95956710652295]

In [15]:
ind = []
default_lat = 35.95956710652295
default_lon = -78.95570271414434
for index,row in enumerate(data):
    #print(data.loc[index,'crash_timestamp'].hour)
    if (data.loc[index,'crash_timestamp'].hour>=20 and data.loc[index,'crash_timestamp'].hour<21
    and data.loc[index,'crash_timestamp'].minute<40) \
    and dist(data.loc[index,'geo_coord_lat'],data.loc[index,'geo_coord_lon'],default_lat,default_lon) <=11:
        print(data.iloc[index,:])


dist b/w (35.90427709037941,-79.0879027517705) and (35.95956710652295,-78.95570271414434) is 13.414857946753399 km
dist b/w (35.84138707447619,-79.14940276678543) and (35.95956710652295,-78.95570271414434) is 21.857046349305932 km
dist b/w (35.912797092182615,-79.09170275262879) and (35.95956710652295,-78.95570271414434) is 13.324271009198625 km
dist b/w (35.90265709127439,-79.05130274012254) and (35.95956710652295,-78.95570271414434) is 10.691201481351913 km
ambulancer                                       Yes
rd_defects                                      None
crsh_sevri                                      None
objectid                                        1803
crash_ty_1                                      None
county                                        Orange
speed_limi                                5 - 15 MPH
rural_urba                                     Urban
crash_mont                                    August
drvr_vehty                             Passenger Car
crash_

## Connect to SQL and create tables

In [16]:
cursor = connect() #Api call to get the cursor

In [17]:
cursor.execute("SELECT version();")
record = cursor.fetchone()
print("You are connected to - ", record,"\n")

You are connected to -  ('PostgreSQL 10.10, compiled by Visual C++ build 1800, 64-bit',) 



### deleting if already existing

In [18]:
query = 'drop schema public cascade;create schema public;'
exec(cursor, query)

### create table crash_details into database traffic_crash

In [19]:
subq=''
for col in data.columns:
    dt = 'INT' if data.loc[:,col].dtype in ['int64','int32'] else 'FLOAT' if data.loc[:,col].dtype in ['float'] else 'TEXT'  
    if col == 'crash_timestamp':
        dt = 'TIMESTAMP'
    if(subq==''):
        subq = '(%s %s'%(col,dt) 
    else:
        
        subq+=',%s %s'%(col,dt) 
    #print(subq)
query = 'create table crash_details%s);'%subq    
print(query)
exec(cursor, query)

create table crash_details(ambulancer TEXT,rd_defects TEXT,crsh_sevri TEXT,objectid TEXT,crash_ty_1 TEXT,county TEXT,speed_limi TEXT,rural_urba TEXT,crash_mont TEXT,drvr_vehty TEXT,crash_type TEXT,city TEXT,workzone_i TEXT,objectid_1 FLOAT,severity TEXT,locality TEXT,crash_grp0 TEXT,crashalcoh TEXT,excsspdind TEXT,rd_feature TEXT,on_rd TEXT,drvr_unitn TEXT,ped_pos TEXT,drvr_injur TEXT,crash_loc TEXT,rd_charact TEXT,drvr_alc_d TEXT,drvr_sex TEXT,ped_race TEXT,light_cond TEXT,drvrage_gr TEXT,ped_unitno TEXT,crashday TEXT,pedage_grp TEXT,ped_alc_dr TEXT,ped_age TEXT,rd_surface TEXT,drvr_race TEXT,crash_sevr TEXT,crash_id TEXT,ped_injury TEXT,developmen TEXT,traff_cntr TEXT,rd_conditi TEXT,region TEXT,rd_class TEXT,geo_coord_lat FLOAT,geo_coord_lon FLOAT,drvr_age TEXT,crsh_typex TEXT,weather TEXT,num_units FLOAT,crash_grp TEXT,num_lanes TEXT,hit_run TEXT,drvr_estsp TEXT,rd_config TEXT,ped_sex TEXT,crash_timestamp TIMESTAMP);


In [20]:
str(data.loc[0,'crash_timestamp'])

'2012-06-09 20:37:00'

### INSERT into Table

In [21]:
d = {}
for rowi in range(data.shape[0]):
    d = data.loc[rowi,:].to_dict()
    col = ','.join(list(d.keys())) # make string for column names
    val = join(data,rowi) #make string for values
    #print(rowi,d)
    query = 'INSERT INTO crash_details(%s) VALUES(%s)'%(col,val)
    #print(query)
    exec(cursor,query) #execute the query

### Count the no of rows in table

In [22]:
exec(cursor,'SELECT * from crash_details;')
record = cursor.fetchall()
len(record)

320

### crashes timing 8 pm and 8:40 pm in any location 
- which is within 11 km radius of coordinates [-78.95570271414434, 35.95956710652295]

In [23]:
#create extension postgis;
query = 'create extension postgis;ALTER TABLE crash_details add column gps geography;'
exec(cursor,query)
#make a new column
query = 'UPDATE crash_details SET gps = ST_SetSRID(ST_MakePoint(geo_coord_lon, geo_coord_lat), 4326);'
exec(cursor,query)
#get the records with in 
query = 'SELECT * FROM crash_details WHERE ST_DWithin(gps, ST_SetSRID(ST_MakePoint(%s,%s), 4326), 11 * 1000) and extract(hour from crash_timestamp) >=20 and extract(hour from crash_timestamp) <21 and extract(minute from crash_timestamp) <40;'%(default_lon,default_lat)
exec(cursor,query)
records = cursor.fetchall()
len(records)


1

#### we can have a for loop to display all the columns in that row and can display in desired format

In [24]:
for r in records:
    for ci in range(len(r)):
        print(r[ci])

Yes
None
None
1803.0
None
Orange
5 - 15 MPH
Urban
August
Passenger Car
Off Roadway - Parking Lot
Chapel Hill
No
16808.0
B: Evident Injury
Urban (>70% Developed)
None
No
No
Driveway, Public
None
1.0
Non-Roadway - Parking Lot / Other
O: No Injury
Non-Roadway
Straight - Level
No
Female
White
Dark - Lighted Roadway
40-49
2.0
Monday
26-30
No
26
Concrete
Black
B: Evident Injury
None
B: Evident Injury
Institutional
No Control Present
Dry
Piedmont
Public Vehicular Area
35.9026570912744
-79.0513027401225
46
38830.0
Clear
2.0
Off Roadway
Unknown
No
0-5 mph
Unknown
Female
2012-08-13 20:39:00
0101000020E610000060C1498B48C353C054437F448AF34140


## simple index

In [26]:
simple_index_query = 'CREATE INDEX crash_gps ON crash_details USING gist(gps);'
exec(cursor,simple_index_query)

## Compsite index

In [27]:
comp_index_query = ' CREATE INDEX crash_index_time_gps ON crash_details (extract(hour from crash_timestamp),extract(minute from crash_timestamp),gps);'
exec(cursor,comp_index_query)

###  Why indexing
when our main file is very big, then the search time for any record in worst case will be 0(no of records), which can be huge, so to speed up we make indexes on basis of some primary/unique key and stores address of records in to that index file. Multilevel index can help in case index file also becomes very large

#### Simple index
on based of 1 column, we can create an index and store the pointers to the database records

#### composite
Based on multiple columns

- now index also consumes a lot of space on disk, so we need to be careful here

Now indexes can be build using B(by default in psql if UNIQUE is used while creating index) or B+ Trees

In [28]:
close()